# JPX Daily Gain Rate Similarity

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
dataF=pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
dataS=pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv')
dataF['class']='train'
dataS['class']='test'
data=pd.concat([dataF,dataS],axis=0)
display(data[0:3])

In [ ]:
data1=data[['Date','SecuritiesCode','Close','class']][data['Date']>'2020-12-31']
print(len(data1))
codes=data1['SecuritiesCode'].unique().tolist()
codesF=data1['SecuritiesCode'][data1['class']=='train'].unique().tolist()
codesS=data1['SecuritiesCode'][data1['class']=='test'].unique().tolist()
print(len(codes),len(codesF),len(codesS))

In [ ]:
data2=pd.DataFrame()
for i,item in enumerate(codes):
    dataitem=data1[data1['SecuritiesCode']==item].reset_index(drop=True)
    dataitem['rate']=(dataitem['Close']-dataitem['Close'].shift(1))/dataitem['Close'].shift(1)
    data2=pd.concat([data2,dataitem[1:]],axis=0)
display(data2)

In [ ]:
data3=data2.dropna()
data3F=data3[data3['class']=='train']
data3S=data3[data3['class']=='test']

In [ ]:
import math
from sklearn.linear_model import LinearRegression

mod = LinearRegression()

def modscore(df_x,df_y):
    R2=mod.score(df_x, df_y)
    return R2

In [ ]:
LIST1=pd.DataFrame(columns=['x','y','R2'],index=list(range(len(codesF)*len(codesS))))

In [ ]:
t=0
for x,itemF in enumerate(codesF[0:10]):
    for y,itemS in enumerate(codesS[0:10]):
        df_x=pd.DataFrame(data3F[data3F['SecuritiesCode']==codesF[x]]['rate'])
        df_y=pd.DataFrame(data3S[data3S['SecuritiesCode']==codesS[y]]['rate'])
        if df_x.isnull().sum()[0]==0 and df_y.isnull().sum()[0]==0 and len(df_x)==225 and len(df_y)==225:
            mod_lin = mod.fit(df_x, df_y)
            y_lin_fit = mod_lin.predict(df_x)
            r2_lin = mod.score(df_x, df_y)
            score=modscore(df_x,df_y)
            LIST1.loc[t,'x']=x
            LIST1.loc[t,'y']=y
            LIST1.loc[t,'R2']=round(score,3) 
            #print(x,y,'wow',round(score,3) )
            t+=1
        else:
            LIST1.loc[t,'x']=x
            LIST1.loc[t,'y']=y
            LIST1.loc[t,'R2']='-'
            #print(x,y,'bad',(len(df_x),len(df_y)))
            t+=1

In [ ]:
LIST1.to_csv('LIST1.csv',index=False)
display(LIST1[0:40])